In [42]:
import os
import shutil
import sys
import json
import argparse
import numpy as np
import math
from einops import rearrange
import time
import random
import string
from tqdm import tqdm
import re
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
from torchvision import transforms
import imageio

device = "cuda"

# tf32 data type is faster than standard float32
torch.backends.cuda.matmul.allow_tf32 = True

In [43]:
def convert_to_snake_case(input_string):
    # Convert the string to lowercase
    lowercased_string = input_string.lower()
    # Remove apostrophes followed by 's'
    no_apostrophe_s = re.sub(r"'s\b", "s", lowercased_string)
    # Remove all other non-alphanumeric characters
    cleaned_string = re.sub(r'[^a-z0-9\s]+', '', no_apostrophe_s)
    # Replace spaces with underscores
    snake_case_string = cleaned_string.replace(" ", "_")
    return snake_case_string

# Example usage
input_string = "Pets', underground, taste's."
output_string = convert_to_snake_case(input_string)
print(output_string) 

pets_underground_tastes


In [44]:
import anthropic
import base64
import os
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

# Initialize the Anthropic client
client = anthropic.Anthropic(
    api_key=os.getenv("ANTHROPIC_API_KEY")
)

def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')

def caption_image(image_path):
    # Encode the image
    base64_image = encode_image(image_path)

    # Create the message with the image
    message = client.messages.create(
        model="claude-3-5-sonnet-20240620",
        max_tokens=100,
        messages=[
            {
                "role": "user",
                "content": [
                    {
                        "type": "image",
                        "source": {
                            "type": "base64",
                            "media_type": "image/jpeg",
                            "data": base64_image
                        }
                    },
                    {
                        "type": "text",
                        "text": "Analyze image and describe it with exactly three words. The three words should allow someone to understand what image was out of a pool of many images. If there is an object in the foreground, describe that instead of the general setting."
                    }
                ]
            }
        ]
    )

    # Return the generated caption
    return message.content[0].text

In [61]:
for ii,filename in enumerate(os.listdir("stimuli")):
    if filename.endswith('.jpg'):
        if "album" in filename:
            image_path = "stimuli/"+filename
            caption = caption_image(image_path)
            
            result = convert_to_snake_case(caption)
            print(ii, result) 
            
            # Move and rename the file
            old_name = image_path.split(".jpg")[0].split("stimuli/")[-1]
            shutil.copy2(image_path, f"stimuli_labeled/{old_name}__{result}.jpg")

16 child_on_tv
19 empire_state_building
200 emotional_fatherdaughter_embrace
271 father_daughter_portrait
314 three_people_smiling
350 colorful_fruit_stand
524 group_of_children
581 man_crouching_outdoors
602 family_gathering_snapshot
624 mustached_man_smiling
627 three_young_girls
658 child_crouching_curiously
699 smiling_couple_portrait
766 colorful_flower_display
773 fruit_store_storefront
811 store_owner_posing
838 fresh_produce_market


In [46]:
# REMOVE 
# coco_nonshared_12610.jpg
# coco_nonshared_9298__magazineembedded_furniture_leg.jpg
# coco_nonshared_34275__antiobama_protest_signs.jpg
# coco_nonshared_54382__dead_animals_dissection.jpg

105

In [63]:
coco_nonshared = 0
coco_shared = 0
extras = 0
targets = 0
albums = 0
for ii,filename in enumerate(os.listdir("stimuli_labeled")):
    if "coco_nonshared" in filename:
        coco_nonshared+=1
    elif "coco_shared" in filename:
        coco_shared+=1
    elif "extra" in filename:
        extras+=1
    elif "target" in filename:
        targets+=1
    elif "album" in filename:
        albums+=1
    # if "dead" in filename:
    #     print(filename)
    # elif "magazineembedded" in filename:
    #     print(filename)
    # elif "obama" in filename:
    #     print(filename)
    # elif ("naked" in filename) or ("nude" in filename):
    #     print(filename)
print("coco_nonshared", coco_nonshared)
print("coco_shared", coco_shared)
print("extras", extras)
print("albums", albums)
print("targets", targets)
print("total", ii)
print("total w repeats", coco_nonshared + (coco_shared*2) + (albums*3) + (extras*3) + (targets*12))

coco_nonshared 379
coco_shared 483
extras 50
albums 17
targets 3
total 932
total w repeats 1582


In [59]:
3000/2.5

1200.0

In [58]:
10/4

2.5